In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!gdown 1f3ToZDiYeym7uXLXtioWGqxtPZu3EHUz
!gdown 10fhw_CIMo7N3XwA0rytnGa_IUcC13GiF
!gdown 1NusXJryZZoqbKErkY5ILGSGo-pzw60Yp

Downloading...
From: https://drive.google.com/uc?id=1f3ToZDiYeym7uXLXtioWGqxtPZu3EHUz
To: /content/processed_test_tweets.txt.zip
100% 289k/289k [00:00<00:00, 149MB/s]
Downloading...
From: https://drive.google.com/uc?id=10fhw_CIMo7N3XwA0rytnGa_IUcC13GiF
To: /content/processed_pos_tweets.txt.zip
100% 30.9M/30.9M [00:00<00:00, 52.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NusXJryZZoqbKErkY5ILGSGo-pzw60Yp
To: /content/processed_neg_tweets.txt.zip
100% 39.0M/39.0M [00:00<00:00, 171MB/s]


In [ ]:
!unzip processed_neg_tweets.txt.zip
!unzip processed_pos_tweets.txt.zip
!unzip processed_test_tweets.txt.zip

Archive:  processed_neg_tweets.txt.zip
  inflating: processed_neg_tweets.txt  
Archive:  processed_pos_tweets.txt.zip
  inflating: processed_pos_tweets.txt  
Archive:  processed_test_tweets.txt.zip
  inflating: processed_test_tweets.txt  


In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate -U
!pip install sentencepiece
!pip install loguru

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.7 MB/s eta 0:00:00


In [ ]:
import logging
import torch
import time
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import pandas as pd
import transformers
import sys
import os
from sklearn.utils import shuffle
from datasets import DatasetDict, Dataset
from sklearn.metrics import accuracy_score, f1_score
from loguru import logger
import pickle

sys.path.append('')

def save_dictionary_as_pickle(dictionary, filename):
    with open(filename, 'wb') as file:
        pickle.dump(dictionary, file)

def load_pickle_as_dictionary(filename):
    with open(filename, 'rb') as file:
        dictionary = pickle.load(file)
    return dictionary

def load_model_from_checkpoint(path_to_checkpoint):
    ''' Helper function, to load the model from a checkpoint.
    takes as input a path to the checkpoint (from the "experiment-[...]" )
     '''
    full_path_to_model_checkpoint = experiment_path + path_to_checkpoint
    model = AutoModelForSequenceClassification.from_pretrained(full_path_to_model_checkpoint, num_labels=2, local_files_only=False, ignore_mismatched_sizes=True)
    print(f"Loaded model from: {full_path_to_model_checkpoint}")
    return model

def numpy_softmax(model_preds):
    '''Converts the raw predictions from a HuggingFace model into clean logits.'''
    max = np.max(model_preds, axis=1, keepdims=True)
    e_x = np.exp(model_preds-max)
    sum = np.sum(e_x, axis=1, keepdims=True)
    out = e_x / sum
    return out

def load_tweets(file_path):
    tweets = list()
    with open(file_path, 'r', encoding='utf-8') as preprocessed_tweets:
        for tweet in preprocessed_tweets:
            tweets.append(tweet.rstrip('\n'))
    return tweets

def preprocess_function(examples, tok_max_length):
    return tokenizer(examples["tweet"], truncation=True, max_length=tok_max_length, padding=True)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

def get_mispredicted_samples(X, y_pred, y_true):
    mispredicted_X = []
    mispredicted_Y = []

    for i in range(len(X)):
        if y_pred[i] != y_true[i]:
            mispredicted_X.append(X[i])
            mispredicted_Y.append(y_true[i])

    return mispredicted_X, mispredicted_Y

def sort_by_difficulty(dataset, dict_path):
    subset_X = dataset['tweet']
    subset_y = dataset['label']
    scores_dict = load_pickle_as_dictionary(dict_path)

    new_dict = {}
    for i, x in enumerate(subset_X):
      new_dict[i] = {'x': x, 'label': subset_y[i], 'score': scores_dict[x]['score']}

    new_sorted_dict = dict(sorted(new_dict.items(), key=lambda item: item[1]['score'], reverse=False))
    sorted_X = [item[1]['x'] for item in new_sorted_dict.items()]
    sorted_y = [item[1]['label'] for item in new_sorted_dict.items()]

    new_data = {"tweet": sorted_X, "label": sorted_y}
    new_dataset = Dataset.from_dict(new_data)
    new_tokenized_dataset = new_dataset.map(lambda examples: preprocess_function(examples, tok_max_length), batched=True)

    return new_tokenized_dataset

def interleave(dataset):
    X_pos_train, y_pos_train = list(zip(*filter(lambda t: t[1] == 1, zip(dataset['tweet'], dataset['label']))))
    X_neg_train, y_neg_train = list(zip(*filter(lambda t: t[1] == 0, zip(dataset['tweet'], dataset['label']))))

    a, b = [X_pos_train, y_pos_train], [X_neg_train, y_neg_train]
    n = len(a[0]) + len(b[0])
    results = []
    for j in range(2):
      true_ratio = len(a[j]) / n
      c = []
      a_count, b_count = 0, 0
      running_ratio = 0
      for i in range(n):
          if running_ratio < true_ratio:
              c.append(a[j][a_count])
              a_count += 1
          else:
              c.append(b[j][b_count])
              b_count += 1
          running_ratio = a_count / (a_count + b_count)
      results.append(c)

    new_data = {"tweet": results[0], "label": results[1]}
    new_dataset = Dataset.from_dict(new_data)
    new_tokenized_dataset = new_dataset.map(lambda examples: preprocess_function(examples, tok_max_length), batched=True)

    return new_tokenized_dataset

# Set default values for the variables
model_name = "vinai/bertweet-base"
batch_size = 32
seed = 12222
fp16 = True
out = "./logging"
epochs = 2
lr = 1e-4
wd = 0.005
tok_max_length = 128
train_val_ratio = 0.99

torch.cuda.empty_cache()
time_run = time.time()

project_path = "./"
experiment_path = "./" + "Experiments/"

experiment_date_for_folder_name = "experiment-" + model_name + "_" + "default"

experiments_results_path = experiment_path + experiment_date_for_folder_name
os.makedirs(experiments_results_path, exist_ok=True)
checkpoints_path = experiments_results_path + "/checkpoints/"
print("The project path is: ", project_path)
print("The experiment path is: ", experiment_path)
print("The model checkpoints will be saved at: ", checkpoints_path, "\n")

# for the submission
test_results_path = experiments_results_path + "/test_results/"
os.makedirs(test_results_path, exist_ok=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device {device}')
np.random.seed(seed)
torch.manual_seed(seed)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

train_pos_tweets = load_tweets('processed_pos_tweets.txt')
#train_pos_tweets = train_pos_tweets[:2000]
train_neg_tweets = load_tweets('processed_neg_tweets.txt')
#train_neg_tweets = train_neg_tweets[:2000]
test_tweets = load_tweets('processed_test_tweets.txt')

#Create labels
train_neg_labels = [0] * len(train_neg_tweets)
train_pos_labels = [1] * len(train_pos_tweets)

train_tweets = train_pos_tweets + train_neg_tweets
train_labels = train_pos_labels + train_neg_labels

#Shuffle
train_tweets, train_labels = shuffle(train_tweets, train_labels, random_state=10)
data = pd.DataFrame({'tweet': train_tweets, 'label': train_labels})

X = list(data["tweet"])
y = list(data["label"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05,stratify=y,random_state=seed)

# Combine X_train and y_train into a single dictionary
train_data = {"tweet": X_train, "label": y_train}
# Convert the dictionary to a Dataset object
train_dataset = Dataset.from_dict(train_data)
# Combine X_val and y_val into a single dictionary
val_data = {"tweet": X_val, "label": y_val}
# Convert the dictionary to a Dataset object
val_dataset = Dataset.from_dict(val_data)

# Combine the train and validation datasets into a DatasetDict
data_dict = DatasetDict({"train": train_dataset, "validation": val_dataset})

#Tokenization using map
tokenized_dataset = data_dict.map(lambda examples: preprocess_function(examples, tok_max_length), batched=True)

#DataCollator for efficient batches
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2,ignore_mismatched_sizes=True).to(device)

logging_steps = 4000
print(logging_steps)
training_args = TrainingArguments(
    output_dir=out,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    save_total_limit=2,
    seed = seed,
    weight_decay=wd,
    evaluation_strategy="epoch",
    gradient_accumulation_steps=4,
    disable_tqdm=False,
    fp16=fp16,
    logging_steps=logging_steps,
    logging_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    load_best_model_at_end=True,
    warmup_steps=500
)

#set here the path
#dictionary_path = '/content/scores_dict.pkl'
#tokenized_dataset_train_new = sort_by_difficulty(tokenized_dataset["train"], dictionary_path)
#tokenized_dataset_train_new = interleave(tokenized_dataset_train_new)

trainer = Trainer(
    model=model,                      # the instantiated HuggingFace model to be trained
    args=training_args,               # training arguments, defined above
    train_dataset=tokenized_dataset["train"],      # training dataset
    eval_dataset=tokenized_dataset["validation"],      # training dataset
    tokenizer=tokenizer,
    data_collator=data_collator,      # data collator
    compute_metrics=compute_metrics,  # metrics function
)

logger.info('Started training')
trainer.train()
logger.info('Ended training')
print(time.time())

data_test = pd.DataFrame({'tweet': test_tweets})
test_dataset = Dataset.from_dict(data_test)
test_dataset = test_dataset.map(lambda examples: preprocess_function(examples, tok_max_length), batched=True)

results = trainer.predict(test_dataset)

y_preds = np.argmax(results.predictions, axis=1)

y_preds = [-1 if val == 0 else 1 for val in y_preds]

df = pd.DataFrame(y_preds, columns=["Prediction"])
df.index.name = "Id"
df.index += 1
df.to_csv(test_results_path+"test_data.csv")

#save also to my drive
test_results_path_drive = '/content/drive/MyDrive/'
df.to_csv(test_results_path_drive + "test_data.csv")

logits = numpy_softmax(results.predictions)

os.makedirs(test_results_path + model_name + "-" + 'logits.txt', exist_ok=True)
np.savetxt(test_results_path  + 'logits.txt', logits, delimiter=",", header = "negative,positive", comments = "")

time_total = time.time() - time_run
print(f"The program took {str(time_total/60/60)[:6]} Hours or {str(time_total/60)[:6]} minutes to run.")

The project path is:  ./
The experiment path is:  ./Experiments/
The model checkpoints will be saved at:  ./Experiments/experiment-vinai/bertweet-base_default/checkpoints/ 

Using device cuda


emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/2153743 [00:00<?, ? examples/s]

Map:   0%|          | 0/113355 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2023-07-19 05:42:07.142 | INFO     | __main__:<cell line: 237>:237 - Started training


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
0,0.259000,0.233110,0.903613,0.903614
1,0.194500,0.226248,0.909073,0.909074


2023-07-19 10:13:31.367 | INFO     | __main__:<cell line: 239>:239 - Ended training


1689761611.36906


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

The program took 4.6244 Hours or 277.46 minutes to run.


In [ ]:
# Save logits_test on Google Drive
with open('/content/drive/MyDrive/logits_test.txt', 'wb') as file:
    pickle.dump(test_results_path + f"logits.txt", file)

In [ ]:
!zip -r /content/Experiments.zip /content/Experiments
!zip -r /content/logging.zip /content/logging
!zip -r /content/logs.zip /content/logs

  adding: content/Experiments/ (stored 0%)
  adding: content/Experiments/experiment-vinai/ (stored 0%)
  adding: content/Experiments/experiment-vinai/bertweet-base_default/ (stored 0%)
  adding: content/Experiments/experiment-vinai/bertweet-base_default/test_results/ (stored 0%)
  adding: content/Experiments/experiment-vinai/bertweet-base_default/test_results/test_data.csv (deflated 66%)
  adding: content/Experiments/experiment-vinai/bertweet-base_default/test_results/vinai/ (stored 0%)
  adding: content/Experiments/experiment-vinai/bertweet-base_default/test_results/vinai/bertweet-base-logits.txt/ (stored 0%)
  adding: content/Experiments/experiment-vinai/bertweet-base_default/test_results/logits.txt (deflated 67%)
  adding: content/logging/ (stored 0%)
  adding: content/logging/checkpoint-18554/ (stored 0%)
  adding: content/logging/checkpoint-18554/optimizer.pt (deflated 24%)
  adding: content/logging/checkpoint-18554/vocab.txt (deflated 51%)
  adding: content/logging/checkpoint-185